In [48]:
from datasets import load_dataset

eli5= load_dataset("eli5", split="train_asks[:5000]") 

Found cached dataset eli5 (C:/Users/AIXI/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


In [49]:
eli5= eli5.train_test_split(test_size=0.2)

In [50]:
eli5["train"][22]

{'q_id': '63x1n1',
 'title': "If you dropped a flea or tick from the Empire State Building, would it die on impact? Or no because of it's respective body mass and size?",
 'selftext': '',
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['dfy61gt', 'dfy6469'],
  'text': ["For any object, you have a terminal velocity depending, to a first approximation, on it's surface area/mass ratio. In general, this will be smaller for smaller objects and so it's more likely that they will survive. \n\n[This](_URL_0_) study from Nature in 1988 suggests that cats falling from about 7 storeys (~100 ft) have a survival rate  > 90%, albeit often with treatable injuries. Since fleas and ticks are say 100x smaller, there is no doubt that they would be able to survive.",
   'No.  Terminal velocity means at some point there will be no further acceleration. The air pushes back and a some point depending on the size of the object,  air resistance equals the force of gravity. \n\nTl:dr for a sm

In [51]:
eli5= eli5.flatten()
eli5["train"][22]

{'q_id': '63x1n1',
 'title': "If you dropped a flea or tick from the Empire State Building, would it die on impact? Or no because of it's respective body mass and size?",
 'selftext': '',
 'document': '',
 'subreddit': 'askscience',
 'answers.a_id': ['dfy61gt', 'dfy6469'],
 'answers.text': ["For any object, you have a terminal velocity depending, to a first approximation, on it's surface area/mass ratio. In general, this will be smaller for smaller objects and so it's more likely that they will survive. \n\n[This](_URL_0_) study from Nature in 1988 suggests that cats falling from about 7 storeys (~100 ft) have a survival rate  > 90%, albeit often with treatable injuries. Since fleas and ticks are say 100x smaller, there is no doubt that they would be able to survive.",
  'No.  Terminal velocity means at some point there will be no further acceleration. The air pushes back and a some point depending on the size of the object,  air resistance equals the force of gravity. \n\nTl:dr for a 

In [52]:
# import pandas as pd


# # Lists to store extracted elements
# titles = []
# selftexts = []
# texts = []

# # Loop through each data point
# for data in eli5['train']:
#     title = data['title']
#     selftext = data['selftext']
#     text = data['answers']['text'][0]

#     titles.append(title)
#     selftexts.append(selftext)
#     texts.append(text)

# # Create a DataFrame
# df = pd.DataFrame({
#     'Title': titles,
#     'Selftext': selftexts,
#     'Text': texts
# })


In [53]:
# df.head()

In [54]:
from transformers import AutoTokenizer
tokenizer= AutoTokenizer.from_pretrained("distilgpt2")

In [59]:
def preprocess_function(examples, tokenizer):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [65]:
tokenized_eli5= eli5.map(
    preprocess_function(eli5["train"], tokenizer),
    # preprocess_function,
    batched=False,
    # num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

TypeError: 'BatchEncoding' object is not callable

In [62]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [63]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

NameError: name 'tokenized_eli5' is not defined

In [40]:
eli5.flatten()

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers.a_id', 'answers.text', 'answers.score', 'title_urls.url', 'selftext_urls.url', 'answers_urls.url'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers.a_id', 'answers.text', 'answers.score', 'title_urls.url', 'selftext_urls.url', 'answers_urls.url'],
        num_rows: 1000
    })
})